Train GRU Tip

    - scaled data : using standarized 、 minmaxize 

    - descise the prediction of lag by using correlation coefficient 

    - the computeation time of the model is so slow , we can use dropout . we need compare dropout 

    - if model have good performation at the training data , we consider using validate data 

    -   

In [ ]:
import pandas as pd

timestep = 10 

timestep_lag_list = []
target_list = []

for i in range(len(data)):
    
    timestep_lag_list.append(data[:i+timestep,:])

    target_list.append(data[i+timestep,:])
    
    entire_data = pd.concat([pd.DataFrame(timestep_lag_list), pd.DataFrame(target_list)], axis=1)


# Create a DataFrame from the list of lists
time_data = pd.DataFrame(entire_data, columns=[f"time_lag_{i+1}" for i in range(timestep)] + ['targettime'])


X = time_data.iloc[:, :-1].values
y = time_data.iloc[:, -1].values


# sp
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)   
 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split



model  = Sequential()
model.add(GRU(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(GRU(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)




